<a href="https://colab.research.google.com/github/Black2oo4g/GoogleColab/blob/main/DATAACOMEMOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Asegurarse que columnas estén bien
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Definir meses y su número
meses_mapping = {
    "ENERO": "01",
    "FEBRERO": "02",
    "MARZO": "03",
    "ABRIL": "04",
    "MAYO": "05",
    "JUNIO": "06",
    "JULIO": "07",
    "AGOSTO": "08",
    "SEPTIEMBRE": "09",
    "OCTUBRE": "10",
    "NOVIEMBRE": "11",
    "DICIEMBRE": "12"
}

# 8. Crear JSON
cols = df_energy.columns.tolist()
json_list = []

for _, row in df_energy.iterrows():
    cod = row["ID"]
    if pd.isna(cod):
        continue
    try:
        cod_int = int(cod)
    except (ValueError, TypeError):
        continue
    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre, mes_num in meses_mapping.items():
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date):
                    continue
                venc_date = emi_date + pd.Timedelta(days=30)

                # El año lo sacamos de emi_date
                year = emi_date.strftime("%Y")

                # El mes es basado en el nombre de la columna
                periodo = f"{year}-{mes_num}"

                json_list.append({
                    "cod": cod_int,
                    "periodo": periodo,
                    "monto": float(monto) if pd.notna(monto) else 0.0,
                    "emi": emi_date.strftime("%Y-%m-%d"),
                    "venc": venc_date.strftime("%Y-%m-%d"),
                    "operacion": str(operacion)
                })

# 9. Guardar JSON
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/salida_pagos.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 11. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo de socios no reconocidos guardado en: {output_unrecognized_path}")

# 12. Mostrar ejemplo de JSON generado
print(json.dumps(json_list[:5], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 254
Total registros NO RECONOCIDOS (sin ID): 169
Archivo de socios no reconocidos guardado en: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx
[
    {
        "cod": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "emi": "2025-04-03",
        "venc": "2025-05-03",
        "operacion": "17399"
    },
    {
        "cod": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "emi": "2025-03-26",
        "venc": "2025-04-25",
        "operacion": "17709"
    },
    {
        "cod": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "emi": "2025-04-23",
        "venc": "2025-05-23",
        "operacion": "18106.0"
    },
    {
        "cod": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "emi": "2025-03-03",
        "venc": "2025-04-02",
       

In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrames de reconocidos y no reconocidos
df_recognized = df_energy.dropna(subset=["ID"])
df_unrecognized = df_energy[df_energy["ID"].isna()]

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]
meses_mapping = {mes: str(i+1).zfill(2) for i, mes in enumerate(meses)}

# 9. Crear JSON agrupado por cod
cols = df_energy.columns.tolist()
json_dict = {}

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    first_emi_found = False
    base_year = None
    mes_idx = 0

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"

                pago = {
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                if cod not in json_dict:
                    json_dict[cod] = []

                json_dict[cod].append(pago)
                mes_idx += 1

# 10. Guardar JSON agrupado
import os
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4, ensure_ascii=False)

# 11. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")

# 13. Mostrar un ejemplo del JSON generado (primer comerciante)
first_cod = next(iter(json_dict))
print(f"Ejemplo de pagos para el COD {first_cod}:")
print(json.dumps(json_dict[first_cod], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 395
Total registros NO RECONOCIDOS (sin ID): 28
Archivo actualizado: /content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx
Archivo de socios no reconocidos: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx
Ejemplo de pagos para el COD 529:
[
    {
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "2025-04-03",
        "operacion": "17399"
    },
    {
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "2025-03-26",
        "operacion": "17709"
    },
    {
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "2025-04-23",
        "operacion": "18106.0"
    }
]


In [21]:
from google.colab import drive
import pandas as pd
import json
import os

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos para todos los comerciantes (sin agrupar)
json_list = []

cols = df_energy.columns.tolist()
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                #periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"
                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2022-{mes_num}"


                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                json_list.append(pago)


# 10. Guardar JSON (sin agrupar por ID)
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2022.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar solo los primeros 5 registros
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
#print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
#print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2022-01",
        "monto": 112.5,
        "fecha_pago": "2022-10-02",
        "operacion": "3436"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-02",
        "monto": 124.5,
        "fecha_pago": "2022-03-15",
        "operacion": "3835"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-03",
        "monto": 165.7,
        "fecha_pago": "2022-05-05",
        "operacion": "4341"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-04",
        "monto": 121.5,
        "fecha_pago": "2022-05-05",
        "operacion": "4341"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-05",
        "monto": 128.4,
        "fecha_pago": "2022-06-20",
        "operacion": "4835"
    },
    {
        "id_comerci

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2024", header=0)
df_target = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021_actualizado.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2024", index=False)
    df_target.to_excel(writer, sheet_name="PAGOS2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428     NAN
Name: Unnamed: 0, Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)
<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428    13.0
Name: , Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Pri

IDs asignados en PAGOS2022:
                               SOCIO MANZANA          ID
0          SILVA SALDARRIAGA ALBERTO      12  1.0  529.0
1          CARRASCO CASTILLO ALFONSO     13A  1.0   18.0
2          VILLEGAS PACHERREZ MANUEL     13B  1.0  621.0
3  HIDALGO VDA. DE CHAPARRO HILBERTA    14-A  1.0   19.0
4  HIDALGO VDA. DE CHAPARRO HILBERTA    14-B  1.0  655.0
